In [ ]:
#Pretend to be a browser.
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36", "X-Requested-With": "XMLHttpRequest"}

#Import needed modules.
import pytz
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from datetime import datetime, date
from dateutil import tz

#Printing out today's date and the current time on the East Coast.
def all_about_time():
    #Prints local date.
    current_time = datetime.now()
    today = date.today()

    local_date = datetime.strftime(today, '%b %d, %Y')
    print('Today\'s date is: '  + str(local_date))
        
    #Prints out the time on the east coast. Helps give context on market hours.    
    eastern_time = datetime.strftime(current_time.astimezone(pytz.timezone('US/Eastern')), '%I:%M %p')
    print('Time on the East Coast is currently: ' + eastern_time)
    
    #This logic block dictates whether the market is closed or open. So far does not account for holidays.
    day_of_week = datetime.strftime(current_time.astimezone(pytz.timezone('US/Eastern')), '%A')
    dt_east = int(datetime.strftime(current_time.astimezone(pytz.timezone('US/Eastern')), '%H%M'))
    if 930 <= dt_east <= 1600 and (day_of_week != 'Saturday' and day_of_week != 'Sunday'):
        print('The market is open!')
    else:
        print('The market is closed.')
        
all_about_time()

#Choose your ticker. Takes input.
ticker = input('Please choose a ticker symbol: ')

#The below try/except will attempt to run everything in "try" as long as the chosen ticker exists. If the ticker does not exist, "except" will run and print the statement that the ticket doesn't exist.

def ticker_results():
  try:
      #Selecting the table to pull data from.
      url = ('https://finviz.com/quote.ashx?t=' + ticker.upper())
      req = requests.get(url, headers = headers)
      table = pd.read_html(req.text, attrs = {'class':'snapshot-table2'})
      df = table[0]
      df2 = table[0]
      
      #Ratios and metrics to be checked.
      data = {
          df[10][10]: ['$' + df[11][10]], #Stock Price
          df[2][0]: [df[3][0]], #P/E - under 20.0
          df[2][2]: [df[3][2]], #PEG - under 1.0
          df[2][4]: [df[3][4]], #P/B - under 1.0
          df[2][6]: [df[3][6]], #P/FCF
          df[0][6]: [df[1][6]], #Annual Dividend
          df[0][7]: [df[1][7]], #Dividend Percent
          df[6][5]: [df[7][5]], #ROE
          'YTD Performance': [df[11][5]]
      }
      
      #Pulls company name.
      soup = BeautifulSoup(req.text, 'html.parser')
      for title in soup.find_all('title'):
          split = title.get_text()
          split2 = split.split()
          company = split2[1] + ' ' + split2[2] + ' ' + split2[3]
          company2 = split2[1] + ' ' + split2[2]
          if split2[3] != 'Stock':
              print('Company/ETF name: ' + company)
          else:
              print('Company/ETF name: ' + company2)
          
      #This will pull the sector of the company.
      req2 = requests.get(url, headers = headers)
      table2 = pd.read_html(req2.text, attrs = {'class':'fullview-title'})
      df3 = table2[0]
      df4 = df3[0][2]
      sector_split = df4.split('|')
      sector = sector_split[0]
      print('Sector: ' + sector)
      
      #Printing title for below table of stats
      print('Metrics:')
      
      #Framing the table and printing the results.
      df = pd.DataFrame(data, index = ["Stats:"])
      display(df)
      
      #pe = float(df2[3][0])
      
      #Website data that is to be scraped.
      EPS_url = ('https://www.alphaquery.com/stock/' + ticker.upper() + '/earnings-history')
      EPS_req = requests.get(EPS_url, headers = headers)
      EPS_table = pd.read_html(EPS_req.text, attrs = {'class':'table table-bordered table-striped table-basic'})
      EPS_df = EPS_table[0]
      
      #Dataset for the EPS annoucement dates.
      dates = [
          EPS_df.iloc[7][0],
          EPS_df.iloc[6][0],
          EPS_df.iloc[5][0],
          EPS_df.iloc[4][0],
          EPS_df.iloc[3][0],
          EPS_df.iloc[2][0],
          EPS_df.iloc[1][0],
          EPS_df.iloc[0][0]
      ]
      
      #Dataset for the EPS values
      EPS_history = [
          EPS_df.iloc[7][3],
          EPS_df.iloc[6][3],
          EPS_df.iloc[5][3],
          EPS_df.iloc[4][3],
          EPS_df.iloc[3][3],
          EPS_df.iloc[2][3],
          EPS_df.iloc[1][3],
          EPS_df.iloc[0][3]
      ]
      
      #Removing the dollar sign from the values and turning them into floats so that MatPlotLib can plot them.
      EPS_history = [dollarsign.strip('$') for dollarsign in EPS_history]
      EPS_history = [float(eps_number) for eps_number in EPS_history]
      
      #Plotting out a graph showcasing EPS history for past eight quarters.
      x = dates
      y = EPS_history
      plt.plot(x, y, 'cd-')
      plt.xlabel('Date')
      plt.xticks(rotation = 45)
      plt.ylabel('EPS')
      plt.title('EPS History - Past Eight Quarters')
      plt.show()
      
      try:
          #Math section which determines how many shares you'll need to buy a whole new share with only dividends being reinvested each quarter.
          def drip():
              price = float(df2[11][10])
              dividend = float(df2[1][6])
              quarter_dividend = dividend / 4
              shares_needed = float('{:.2f}'.format(price / quarter_dividend))
              cost = '{:,.2f}'.format(price * shares_needed)
      
              print('With quarterly dividend reinvestment, you\'ll need about ' + str(shares_needed) + ' shares to purchase a new share with only dividends. This amount of shares will cost about $' + str(cost))
          
          drip()
          
      except:
          print('Stock does not pay a dividend.')
      
  except ValueError:
      print('Ticker doesn\'t exist. Please check your selection.')

ticker_results()